In [2]:
import pandas as pd
from datetime import timedelta

In [3]:
# setear parametros/variables
p_mes = "diciembre"
input_file = "202012"

In [4]:
csv_columns = ['PRIMARIO', 'FUERA DE SERVICIO', 'REPOSICION', 'TIPO DE EVENTO', 'ESTADO DEL CLIMA', 'OBSERVACION']

In [5]:
# leer archivos
df1 = pd.read_csv(fr"../data/{input_file}.csv", encoding="ISO-8859-1")

In [6]:
# Transformar datasets
df1n = df1.BARRIOS.str.rstrip(',').str.rstrip(',.').str.split(',').apply(pd.Series)    # break df.BARRIOS into columns
df1n.index = df1.set_index(csv_columns).index
df1 = df1n.stack().reset_index(csv_columns)    # stack and reset_index
df1.columns = csv_columns  + ['BARRIO']

In [7]:
# Considerar Recierre como 2 horas
def transform_recierre(fuera_servicio, reposicion):
#     print("fuera servicio:", fuera_servicio)
#     print("reposicion:", reposicion)
    if reposicion.lower().strip() != "recierre":
        return reposicion
    
    reposicion = pd.to_datetime(fuera_servicio, format="%d/%m/%Y %H:%M", errors='coerce') + timedelta(hours=2)
    # print(reposicion.strftime("%d/%m/%Y %H:%M"))
    return reposicion.strftime("%d/%m/%Y %H:%M")

df1['REPOSICION'] = df1.apply(lambda row: transform_recierre(row['FUERA DE SERVICIO'], row['REPOSICION']), axis=1)

In [8]:
df1['DURACION'] = df1[["FUERA DE SERVICIO", "REPOSICION"]].apply(lambda x: (pd.to_datetime(x[1], format="%d/%m/%Y %H:%M", errors='coerce') - pd.to_datetime(x[0], format="%d/%m/%Y %H:%M", errors='coerce')), axis=1)

In [9]:
# Normalizar BARRIO
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\r", " "))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("\n", " "))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("  ", " "))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.strip())
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Hiódromo", "Hipódromo"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Madame Lych", "Madame Lynch"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Dr. Gaspar R. de Francia", "Dr. Gaspar Rodríguez de Francia"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Rodriguez", "Rodríguez"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. Francisco.S. López", "Mcal. Francisco S. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace('Mcal F. S. López', "Mcal. Francisco S. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace('Francisco.', 'Francisco'))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mbocayty", "Mbocayaty"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mobocayaty", "Mbocayaty"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cañada de Ybyray", "Cañada Ybyray"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazaret", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Nazarethh", "Nazareth"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Carlos. A. López", "Carlos A. López"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Petirosi", "Pettirossi"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Cristobal", "Cristóbal"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Santisima", "Santísima"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mocayaty", "Mbocayaty"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("Mcal. José F. Estigarribia","Mcal. Estigarribia"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.replace("ybyray", "Ybyray"))
df1['BARRIO'] = df1['BARRIO'].apply(lambda x: x.rstrip("."))


In [10]:
## Normalizar PRIMARIO
df1['PRIMARIO'] = df1['PRIMARIO'].apply(lambda x: x.rstrip("."))

In [11]:
# Agregar Duración en Segundos
df1['DURACION SEGS'] = df1['DURACION'].apply(lambda x: x.seconds)

# Verificar datos

In [12]:
# no deben repetirse barrios
aux = sorted(list(df1["BARRIO"].unique()))
aux

['Banco San Miguel',
 'Bañado Cará Cará',
 'Bañado Tacumbu',
 'Bella Vista',
 'Botánico',
 'Carlos A. López',
 'Cañada Ybyray',
 'Ciudad Nueva',
 'Dr. Gaspar Rodríguez de Francia',
 'Dr. Roberto L. Petit',
 'Gral. Bernardino Caballero',
 'Gral. J. E Diaz',
 'Hipódromo',
 'Ita Pyta Punta',
 'Itay',
 'Itá Enramada',
 'Jara',
 'Jukyty',
 'La Catedral',
 'La Encarnación',
 'Las Carmelitas',
 'Las Mercedes',
 'Loma Pyta',
 'Los Laureles',
 'Luis Alberto de Herrera',
 'Madame Lynch',
 'Manorá',
 'Mbocayaty',
 'Mburicao',
 'Mburucuyá',
 'Mcal. Estigarribia',
 'Mcal. Francisco S. López',
 'Obrero',
 'Pinoza',
 'Pirizal',
 'Recoleta',
 'Republicano',
 'Ricardo Brugada',
 'Sajonia',
 'Salvador del Mundo',
 'San Antonio',
 'San Blas',
 'San Cristóbal',
 'San Felipe',
 'San Jorge',
 'San Pablo',
 'San Roque',
 'San Vicente',
 'Santa Ana',
 'Santa Librada',
 'Santa María',
 'Santa Rosa',
 'Santo Domingo',
 'Santísima Trinidad',
 'Tablada Nueva',
 'Tembetary',
 'Terminal',
 'Tte. Silvio Pettirossi',

In [13]:
# debe haber 69 o 68 barrios como max
len(aux)

65

In [14]:
# Revisar que no queden recierres
df1.loc[(df1["REPOSICION"]=="Recierre")]

,PRIMARIO,FUERA DE SERVICIO,REPOSICION,TIPO DE EVENTO,ESTADO DEL CLIMA,OBSERVACION,BARRIO,DURACION,DURACION SEGS


In [20]:
# Exportar dataframe transformado
df1.to_csv(f"../transformed/t_{p_mes}.csv", encoding="ISO-8859-1")

In [15]:
df1

,PRIMARIO,FUERA DE SERVICIO,REPOSICION,TIPO DE EVENTO,ESTADO DEL CLIMA,OBSERVACION,BARRIO,DURACION,DURACION SEGS
0,BCP2,16/12/2020 08:02,16/12/2020 10:02,Accidental,Normal,No establecido.,Bella Vista,0 days 02:00:00,7200
1,BCP2,16/12/2020 08:02,16/12/2020 10:02,Accidental,Normal,No establecido.,Santo Domingo,0 days 02:00:00,7200
2,BCP2,16/12/2020 08:02,16/12/2020 10:02,Accidental,Normal,No establecido.,Manorá,0 days 02:00:00,7200
0,BCP2,16/12/2020 08:05,16/12/2020 10:05,Accidental,Normal,No establecido.,Bella Vista,0 days 02:00:00,7200
1,BCP2,16/12/2020 08:05,16/12/2020 10:05,Accidental,Normal,No establecido.,Santo Domingo,0 days 02:00:00,7200
...,...,...,...,...,...,...,...,...,...
0,VAU5,19/12/2020 23:06,20/12/2020 01:06,Accidental,Adverso,Condiciones Climáticas Adversas.,Hipódromo,0 days 02:00:00,7200
1,VAU5,19/12/2020 23:06,20/12/2020 01:06,Accidental,Adverso,Condiciones Climáticas Adversas.,Mcal. Estigarribia,0 days 02:00:00,7200
2,VAU5,19/12/2020 23:06,20/12/2020 01:06,Accidental,Adverso,Condiciones Climáticas Adversas.,Villa Aurelia,0 days 02:00:00,7200
0,VAU6,3/12/2020 03:01,3/12/2020 03:14,Accidental,Normal,Abierto interruptor para efectuar reparación d...,Mcal. Estigarribia,0 days 00:13:00,780


In [17]:
aux4 = df1.sort_values(by=['BARRIO'], ascending=False)
aux4.loc[aux4.BARRIO == ""]

,PRIMARIO,FUERA DE SERVICIO,REPOSICION,TIPO DE EVENTO,ESTADO DEL CLIMA,OBSERVACION,BARRIO,DURACION,DURACION SEGS


In [18]:
AUX = df1.groupby(['TIPO DE EVENTO', 'BARRIO'])['BARRIO'].count()

In [19]:
AUX[["Accidental"]].sort_values(ascending=False)

TIPO DE EVENTO  BARRIO          
Accidental      Ita Pyta Punta      17
                Mburucuyá           16
                Madame Lynch        14
                Pirizal             13
                Pinoza              13
                                    ..
                San Cristóbal        2
                Jara                 1
                Tembetary            1
                Tablada Nueva        1
                Banco San Miguel     1
Name: BARRIO, Length: 64, dtype: int64

In [ ]:
AUX[["Programado"]].sort_values(ascending=False)

In [ ]:
aux3 = df1[['BARRIO', 'DURACION']]

In [ ]:
aux4 = df1.sort_values(by=['DURACION'], ascending=False)

In [ ]:
aux4.sort_values(by=["DURACION"], ascending=False)

In [ ]:
aux4.loc[(aux4["REPOSICIONN"]=="Recierre") & (aux4["FUERA DE SERVICIO"].str.contains("10/01/2020"))]

In [ ]:
aux4.loc[(aux4["FUERA DE SERVICIO"].str.contains("31/01/2020"))]

In [ ]:
tdelta2 = pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M") - pd.to_datetime("6/01/2020 23:54", format="%d/%m/%Y %H:%M")
tdelta2

In [ ]:
pd.to_datetime("7/01/2020 01:12", format="%d/%m/%Y %H:%M")